In [11]:
import pandas as pd
from prophet import Prophet
from openpyxl import load_workbook
from openpyxl.styles import Font, PatternFill
from openpyxl.formatting.rule import CellIsRule

import sys
import os

sys.path.append(os.path.abspath(".."))
from src.alert_generator import generate_stock_alerts
from src.data_loader import load_inventory_data

# Load data
df = load_inventory_data("../data/retail_store_inventory_data.xlsx")

# Forecast demand for a specific product/store
product_id = "P0019"
store_id = "S005"
filtered = df[(df["Product ID"] == product_id) & (df["Store ID"] == store_id)][["Date", "Units Sold"]]
filtered = filtered.rename(columns={"Date": "ds", "Units Sold": "y"})

# Train Prophet model
model = Prophet()
model.fit(filtered)

# Make future predictions (14 days ahead)
future = model.make_future_dataframe(periods=14)
forecast = model.predict(future)[["ds", "yhat", "yhat_lower", "yhat_upper"]]

# Save forecasts and alerts to Excel first
with pd.ExcelWriter("../data/output_forecast.xlsx", engine="openpyxl") as writer:
    forecast.to_excel(writer, sheet_name="Forecast", index=False)
    alerts_df = generate_stock_alerts(df)
    alerts_df.to_excel(writer, sheet_name="LowStockAlerts", index=False)

# Load workbook to apply formatting
wb = load_workbook("../data/output_forecast.xlsx")

# Format 'Forecast' sheet
ws_forecast = wb["Forecast"]
header_font = Font(bold=True)
for cell in ws_forecast[1]:
    cell.font = header_font

# Format 'LowStockAlerts' sheet
ws_alerts = wb["LowStockAlerts"]
for cell in ws_alerts[1]:
    cell.font = header_font

# Apply conditional formatting: highlight Inventory Level < 20 in red fill
red_fill = PatternFill(start_color="FFC7CE", end_color="FFC7CE", fill_type="solid")
ws_alerts.conditional_formatting.add(
    "C2:C100",  # Inventory Level column (assumes <100 rows)
    CellIsRule(operator="lessThan", formula=["20"], fill=red_fill)
)

# Auto column width
for sheet in [ws_forecast, ws_alerts]:
    for col in sheet.columns:
        max_length = max(len(str(cell.value)) if cell.value is not None else 0 for cell in col)
        sheet.column_dimensions[col[0].column_letter].width = max_length + 2

# Save formatted workbook
wb.save("../data/output_forecast.xlsx")
print("🎯 Excel formatting applied. File ready at: data/output_forecast.xlsx")

09:51:51 - cmdstanpy - INFO - Chain [1] start processing
09:51:51 - cmdstanpy - INFO - Chain [1] done processing


🎯 Excel formatting applied. File ready at: data/output_forecast.xlsx


In [12]:
from openpyxl.chart import LineChart, Reference

# Reopen workbook
wb = load_workbook("../data/output_forecast.xlsx")
ws = wb["Forecast"]

# Create line chart
chart = LineChart()
chart.title = "Forecasted Demand"
chart.y_axis.title = "Predicted Units Sold"
chart.x_axis.title = "Date"

# Set data range (yhat values)
data = Reference(ws, min_col=2, min_row=1, max_row=ws.max_row)  # yhat
cats = Reference(ws, min_col=1, min_row=2, max_row=ws.max_row)  # ds (dates)
chart.add_data(data, titles_from_data=True)
chart.set_categories(cats)

# Add chart to sheet
ws.add_chart(chart, "E2")

# Save with chart
wb.save("../data/output_forecast.xlsx")
print("📈 Line chart added to Forecast sheet.")

📈 Line chart added to Forecast sheet.
